<a href="https://colab.research.google.com/github/cortiz313/Machine-Learning-Class/blob/main/projects/Bach_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bach Project

By: Christian Ortiz

## Setting up the Environment

In [ ]:
from pandas import DataFrame, Series
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
from sklearn.model_selection import GridSearchCV

## Reading the Data

In [ ]:
!wget https://raw.githubusercontent.com/zacharski/ml-class/master/data/bach.zip

--2023-03-07 21:34:56--  https://raw.githubusercontent.com/zacharski/ml-class/master/data/bach.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41761 (41K) [application/zip]
Saving to: ‘bach.zip’

bach.zip            100%[===================>]  40.78K  --.-KB/s    in 0.003s  

2023-03-07 21:34:56 (11.9 MB/s) - ‘bach.zip’ saved [41761/41761]



## Bagging Method

In [ ]:
bach = pd.read_csv('bach.zip')

In [ ]:
bach

,choral_ID,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
0,000106b_,1,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
1,000106b_,2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5,C_M
2,000106b_,3,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2,C_M
3,000106b_,4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
4,000106b_,5,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,015505b_,105,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4,G_m
5661,015505b_,106,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3,G_m
5662,015505b_,107,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5,C_M
5663,015505b_,108,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,C,3,C_M


#### Removing columns

In [ ]:
bach.drop(columns='choral_ID', inplace=True)
bach

,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
0,1,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
1,2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5,C_M
2,3,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2,C_M
3,4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
4,5,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,105,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4,G_m
5661,106,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3,G_m
5662,107,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5,C_M
5663,108,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,C,3,C_M


In [ ]:
bFeatures = bach.drop('chord_label', axis=1)
bLabels = bach['chord_label']

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
bachOneHot = enc.fit_transform(bFeatures)
bachOneHot

<5665x252 sparse matrix of type '<class 'numpy.float64'>'
	with 84975 stored elements in Compressed Sparse Row format>

In [ ]:
bach_train_features, bach_test_features, bach_train_labels, bach_test_labels = train_test_split(bachOneHot, bLabels, test_size = 0.2, random_state=42)

In [ ]:
clf = tree.DecisionTreeClassifier(criterion='entropy')

In [ ]:
param_grid = {'n_estimators' : [5, 10, 20, 50, 100], 'max_samples': [10, 20, 50, 100, 200]}

In [ ]:
bagging_clf = BaggingClassifier(clf, n_estimators=20, max_samples=100, 
                                bootstrap=True, n_jobs=-1)

In [ ]:
grid_search = GridSearchCV(bagging_clf, param_grid, cv = 10) 

In [ ]:
%%time
grid_search.fit(bach_train_features, bach_train_labels)

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


CPU times: user 11.8 s, sys: 3.55 s, total: 15.3 s
Wall time: 31.3 s


GridSearchCV(cv=10,
             estimator=BaggingClassifier(estimator=DecisionTreeClassifier(criterion='entropy'),
                                         max_samples=100, n_estimators=20,
                                         n_jobs=-1),
             param_grid={'max_samples': [10, 20, 50, 100, 200],
                         'n_estimators': [5, 10, 20, 50, 100]})

In [ ]:
grid_search.best_params_

{'max_samples': 200, 'n_estimators': 50}

In [ ]:
predictions = grid_search.predict(bach_test_features)
acc = accuracy_score(bach_test_labels, predictions)

In [ ]:
acc

0.703442188879082

#### Best Accuracy Score I Achieved with Bagging:

*   70.34%



## XGBoost Method

In [ ]:
bach = pd.read_csv('bach.zip')

In [ ]:
bach

,choral_ID,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
0,000106b_,1,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
1,000106b_,2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5,C_M
2,000106b_,3,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2,C_M
3,000106b_,4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
4,000106b_,5,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,015505b_,105,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4,G_m
5661,015505b_,106,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3,G_m
5662,015505b_,107,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5,C_M
5663,015505b_,108,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,C,3,C_M


In [ ]:
bach.drop(columns='event_number', inplace=True)
bach.drop(columns='choral_ID', inplace=True)
bach

,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
0,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
1,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5,C_M
2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2,C_M
3,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4,G_m
5661,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3,G_m
5662,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5,C_M
5663,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,C,3,C_M


In [ ]:
counts = bach['chord_label'].value_counts()

# select only the rows where the value appears once
keep_values = counts[counts < 2].index.tolist()
bach_one_instance = bach[bach['chord_label'].isin(keep_values)]

bach_one_instance

,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
771,NO,NO,YES,YES,NO,NO,NO,YES,NO,NO,NO,NO,D,5,EbM7
3076,YES,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,NO,F#,3,F#d7
3114,NO,NO,YES,NO,NO,NO,NO,NO,YES,NO,NO,YES,B,3,Abd
3188,NO,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,NO,F#,4,Ebd
3464,NO,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,YES,F,4,DbM7
4652,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,NO,YES,F,3,F_d7
4782,YES,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,NO,G#,3,G#M
5159,YES,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,NO,D#,4,D#d6


In [ ]:
# drop values that only appear once
bach = bach[~bach.isin(bach_one_instance)].dropna()
bach

,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
0,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3.0,F_M
1,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5.0,C_M
2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2.0,C_M
3,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3.0,F_M
4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2.0,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4.0,G_m
5661,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3.0,G_m
5662,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5.0,C_M
5663,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,C,3.0,C_M


In [ ]:
bFeatures = bach.drop('chord_label', axis=1)
bLabels = bach['chord_label']

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
bachOneHot = enc.fit_transform(bFeatures)
bachOneHot

<5657x45 sparse matrix of type '<class 'numpy.float64'>'
	with 79198 stored elements in Compressed Sparse Row format>

In [ ]:
le = LabelEncoder()
bach_Labels = le.fit_transform(bLabels)
bach_Labels

array([75, 34, 34, ..., 34, 34, 75])

In [ ]:
bach_train_features, bach_test_features, bach_train_labels, bach_test_labels = train_test_split(bachOneHot, bach_Labels, test_size = 0.2, random_state=42, stratify=bach_Labels)

In [ ]:
from xgboost import XGBClassifier
params = { 'tree_method':'gpu_hist', 'predictor':'gpu_predictor'}

model = XGBClassifier(**params)
model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor='gpu_predictor', random_state=None, ...)

In [ ]:
param_grid = {'n_estimators' : [50, 100, 150, 200, 400], 'max_depth' : [1,2,4,6,8,10]}

In [ ]:
param_comb = 5
folds=5
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=param_comb,  n_jobs=-1, 
                                   cv=skf.split(bach_train_features,bach_train_labels), verbose=3)

In [ ]:
%%time 
search_result = random_search.fit(bach_train_features, bach_train_labels)

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Fitting 5 folds for each of 5 candidates, totalling 25 fits


/usr/local/lib/python3.8/dist-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


CPU times: user 14.6 s, sys: 1.16 s, total: 15.8 s
Wall time: 28min


In [ ]:
random_search.best_params_

{'n_estimators': 100, 'max_depth': 4}

In [ ]:
search_predictions = random_search.best_estimator_.predict(bach_test_features)

In [ ]:
search_acc = accuracy_score(bach_test_labels, search_predictions)
search_acc

0.7340989399293286

#### Best Accuracy Score I Achieved with XGBoost:

*   73.41%

## Final Results

### The best accuracy I was able to achieve, from either XGBoost or Bagging, was 73.41%. That accuracy came from XGBoost, with hyperparameters of n_estimators = 100 and max_depth = 4. I tried my best to get the accuracy above 74%, but neither the parameters I experimented with, nor the methods I tried (XGBoost, bagging, pasting, random subspaces) allowed me to get a better result. It was still enjoyable trying to get the highest score I could.